In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Brownian Bridge Simulation and Metric Estimation on Lie Groups and Homogeneous Spaces

Stefan Sommer and Sarang Joshi

In [ ]:
%cd ..
# SO(3)
from src.groups.SON import *
G = SON(3,invariance='right')
print(G)

# SO(3) acts on S^2
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *

In [ ]:
from src.group import invariant_metric
invariant_metric.initialize(G)

from src.group import energy
energy.initialize(G)

In [ ]:
q = np.array([1e-6,0,0])
g = G.psif(q)
v = np.array([0,1,1])
p = G.sharppsif(q,v)

In [ ]:
# sample data
from src.stochastics import Brownian_inv
Brownian_inv.initialize(G)

G.sigma.set_value(np.diag((1.,.3,1.4))) # set metric
K = 1024
obss = np.zeros((K,)+g.shape)
# srng.seed(422)
for i in range(K):
    (ts,gs) = G.Brownian_invf(g,dWsf(G.dim.eval()))
    obss[i] = gs[-1]

# plot samples
newfig()
for i in range(K):
    G.plotg(obss[i])
plt.show()

# on S2
x = np.array([0,0,1])
obss_M = np.zeros((K,M.emb_dim.eval(),))
newfig()
M.plot()
for i in range(K):
    obss_M[i] = M.actf(obss[i],x)
    M.plotx(obss_M[i])
plt.show()

In [ ]:
# Fisher-Bingham, using https://github.com/edfraenkel/kent_distribution
import sys
sys.path.append("kent_distribution")
from kent_distribution import *

# # k = kent(0.0,       0.0,     0.0,    1.0,  0.0)
# #print("Original Distribution: k =", k)
# gridsize = 200
# u = linspace(0, 2 * pi, gridsize)
# v = linspace(0, pi, gridsize)

# x = outer(cos(u), sin(v))
# y = outer(sin(u), sin(v))
# z = outer(ones(size(u)), cos(v))

# keys = list()
# points = list()
# for i in range(gridsize):
#   for j in range(gridsize):
#     points.append([x[i, j], y[i, j], z[i, j]])
#     keys.append((i, j))
# points = array(points)

#print("Drawing 10000 samples from k")
#xs = k.rvs(10000)
xs = obss_M
k_me = kent_me(xs)
print("Moment estimation:  k_me =", k_me)
k_mle = kent_mle(xs, warning=sys.stdout)
print("Fitted with MLE:   k_mle =", k_mle)
assert k_me.log_likelihood(xs) < k_mle.log_likelihood(xs)

# value_for_color = k_mle.pdf(points)
# value_for_color /= max(value_for_color)  
# colors = empty((gridsize, gridsize), dtype=tuple)
# for (i, j), v in zip(keys, value_for_color):
#   colors[i, j] = (1.0-v, 1.0-v, 1.0, 1.0)

# f = plt.figure()
# ax = f.add_subplot(111, projection='3d')
# xx, yy, zz = zip(*xs[:100]) # plot only a portion of these values
# ax.scatter(1.05*array(xx), 1.05*array(yy), 1.05*array(zz), c='b')
# ax.plot_surface(x, y, z, rstride=4, cstride=4, facecolors=colors, linewidth=0)

newfig()
plot_sphere_f(M, k_mle.pdf,alpha=.8,vmin=0.,colorbar=True)
# plt.savefig('sphere_kent.pdf') # kent distribution
# plt.axis('off')
plt.show()

In [ ]:
# plot estimated density, 
newfig()
# plotM(alpha=.4)
plot_sphere_density_estimate(M, obss_M,pts=100,alpha=.8,bandwidth=.15) # spherical coordinates
# plot_density_estimate(obss_M,limits=[-3,3,-3,3],pts=500,alpha=.4,bandwidth=.15) # general ellipsoidal coordinates (note: very long computation time)
# for i in range(min(512,K)):
#     obss_M[i] = actf(obss[i],x0)
#     plotx(obss_M[i],color='k',s=10)
# plt.savefig('sphere_anistropic-density_est.pdf')
plt.show()

In [ ]:
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# parameters
g0 = G.sym_element()
thetas = (g0, G.sigma,)
thetas_true = [g]+[theta.eval() for theta in thetas[1:]]

# guide function
phi = lambda g,v: G.LAtoV(G.invtrns(G.inv(g),v)-G.e)
# phi = lambda g,v: T.tensordot(G.inv(G.sigma),G.LAtoV(G.log(G.invtrns(G.inv(g),v))),(1,0))

# (Brownian_inv_guided,Brownian_inv_guidedf) = get_guided_likelihood(G,G.sde_Brownian_inv,phi,lambda g: G.sigma, g0, thetas, A=G.gG, integration='stratonovich')
(Brownian_inv_guided,Brownian_inv_guidedf) = get_guided_likelihood(
    G,G.sde_Brownian_inv,phi,lambda g: G.sigma, 
    A=G.gG, integration='stratonovich')

w = G.psif(v)
(ts,gs,log_likelihood,log_varphi) = Brownian_inv_guidedf(g,w,dWsf(G.dim.eval()))[:4]
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

newfig()
G.plot_path(gs)
G.plotg(w,color='k')
# plt.savefig('SO3-bridge.pdf')
plt.show()

# on S2
newfig()
M.plot()
M.plot_path(M.actsf(gs.transpose((1,2,0)),x).T)
M.plotx(M.actf(w,x),color='k',s=60)
# plt.savefig('S2-bridge.pdf')
plt.show()

In [ ]:
# fiber Brownian motion
from src.group.quotient import *
sde_Brownian_inv_fiber = get_sde_fiber(G.sde_Brownian_inv,lambda g: M.act(g,x),G,M)
Brownian_inv_fiber = lambda g,dWt: integrate_sde(sde_Brownian_inv_fiber,
                                                  integrator_stratonovich,
                                                  None,
                                                 g,None,dWt)
Brownian_inv_fiberf = theano.function([g0,dWt], Brownian_inv_fiber(g0,dWt))

# make L simulations
L = 10
gsl = np.zeros((L,)+g.shape)
vl = G.psif(v)
for l in range(L):
    (ts,gs) = Brownian_inv_fiberf(vl,dWsf(G.dim.eval()))[:2]
    gsl[l] = gs[-1]
    vl = gs[-1]
newfig()
step = 5
for l in range(0,L,step):
    (ts,gs,log_likelihood,log_varphi) = Brownian_inv_guidedf(g,gsl[l],dWsf(G.dim.eval()))[:4]
    G.plot_path(gs)
    G.plotg(gsl[l],color='k')
# plt.savefig('SO3-bridges_fiber.pdf')
plt.show()

# on S2
newfig()
M.plot()
for l in range(0,L,step):
    (ts,gs,log_likelihood,log_varphi) = Brownian_inv_guidedf(g,gsl[l],dWsf(G.dim.eval()))[:4]
    M.plot_path(M.actsf(gs.transpose((1,2,0)),x).T)
    M.plotx(M.actf(gsl[l],x),color='k',s=60)
#     plt.savefig('S2-bridges_fiber.pdf')
plt.show()

In [ ]:
# estimate mass of SO(2) fibers in SO(3)
def lfiber_samples(*args, **kwargs):
    return partial(fiber_samples,G,Brownian_inv_fiberf)(*args,**kwargs)
fiber_volume = estimate_fiber_volume(G,M,lfiber_samples,nr_samples=400,plot_dist_histogram=True,plot_samples=True)
print("estimated fiber volume: %f" % fiber_volume)

In [ ]:
options = {}
options['samples_per_obs'] = 20
options['epochs'] = 10
options['learning_rate'] = 6.e-3
options['varphi_update_rate'] = 1.
options['verbose'] = True
options['initial'] = [obss[0], # random value
                      np.diag((.08,.1,.2)),]
# options['update_v'] = lambda g: theano.gradient.disconnected_grad(Brownian_inv_fiber(g,dWs(G.dim))[1][-1])
# options['update_vf'] = lambda g: Brownian_inv_fiberf(g,dWsf(G.dim.eval()))[1][-1]

In [ ]:
# Transition density
v0 = G.sym_element()
p_Tf = theano.function([g0,v0],p_T(g0,v0,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv,sigma=G.sigma))
log_p_Tf = theano.function([g0,v0],log_p_T(g0,v0,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv,sigma=G.sigma))
dlog_p_Tf = theano.function([g0,v0],dlog_p_T(thetas,g0,v0,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv,sigma=G.sigma))

# G.sigma.set_value(np.diag((1.,.3,1.6))) # set metric

# # on G
print(p_Tf(g,G.psif(v))) 
print(log_p_Tf(g,G.psif(v))) 
print(dlog_p_Tf(g,G.psif(v)))

newfig()
C_fiber = 2*np.pi # mass of fiber w.r.t. base measure on SO(3) from biinvariant metric
f = lambda v: np.sqrt(C_fiber)*G.sigma.eval()[0,0]*p_Tf(g,lift_to_fiber(v,x,G,M)[0])
plot_sphere_f(M,f,alpha=.8,pts=100,parallel=True,vmin=0.)
# plt.savefig('sphere_pT.pdf') # transition density plot
plt.show()

In [ ]:
%%time
print(p_Tf(g,G.psif(v))) 

In [ ]:
%%time
print(dlog_p_Tf(g,G.psif(v)))

In [ ]:
%%time
L = options['samples_per_obs']
gsl = np.zeros((L,)+g.shape)
vl = G.psif(v)
for l in range(L):
#     (ts,gs) = Brownian_inv_fiberf(vl,dWsf(G.dim.eval()))
#     (ts,gs) = G.Brownian_invf(vl,dWsf(G.dim.eval()))
    (ts,gs,_,_) = Brownian_inv_guidedf(vl,w,dWsf(G.dim.eval()))
    gsl[l] = gs[-1]
    vl = gs[-1]

In [ ]:
# samples for MLE
# G.sigma.set_value(1.*np.eye(G.dim.eval())) # set metric, uniform
G.sigma.set_value(np.diag((.2,.2,1.5))) # anisotropic
K = 64

obss = np.zeros((K,)+g.shape)
# srng.seed(422)
for i in range(K):
    (ts,gs) = G.Brownian_invf(g,dWsf(G.dim.eval()))
    obss[i] = gs[-1]

# plot samples
newfig()
for i in range(K):
    G.plotg(obss[i])
plt.show()

# on S2
x = np.array([0,0,1])
obss_M = np.zeros((K,M.emb_dim.eval(),))
newfig()
M.plot()
for i in range(K):
    obss_M[i] = M.actf(obss[i],x)
    M.plotx(obss_M[i],color='k')
plt.show()

In [ ]:
# sample bridges
def lbridge_sampling(thetas,*args,**kwargs):
    g = thetas[0]
    G.sigma.set_value(thetas[1])
    return partial(bridge_sampling,g,Brownian_inv_guidedf,lambda: dWsf(G.dim.eval()),options)(*args,**kwargs)

log_phis = np.zeros((K,))
try:
    mpu.openPool()
    sol = mpu.pool.imap(partial(lbridge_sampling,options['initial']),mpu.inputArgs(obss,np.random.randint(1000,size=K)))
    res = list(sol)
    bridges = mpu.getRes(res,0)
    log_varphis = mpu.getRes(res,1)
    log_likelihoods = mpu.getRes(res,2)
except:
    mpu.closePool()
    raise
else:
    mpu.closePool()

# Plot on S2
newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, K)]
for i in range(bridges.shape[0]):
    M.plotx(M.actf(obss[i],x),color=colors[i])
    for j in range(bridges.shape[1]):
        gs = bridges[i,j]
        M.plotx(M.actsf(gs.transpose((1,2,0)),x).T,linewidth=.2,color=colors[i])
# plt.savefig('sphere_samples_with_bridges_uniform.pdf') # uniform
# plt.savefig('sphere_samples_with_bridges_anisotropic.pdf') # anisotropic
plt.show()

In [ ]:
%%time
# on SO(3)
from src.statistics.mle import *

def llog_p_T(thetas,pars):
    (v,seed) = pars
    if seed:
        srng.seed(seed)
    g = thetas[0]
    G.sigma.set_value(thetas[1])
    return dlog_p_Tf(g,v)

def update_thetas(thetas, dthetas):
    g = thetas[0]
    sigma = thetas[1]
    
#     g = G.to_groupf(g+options['learning_rate']*dthetas[0])
    sigma += options['learning_rate']*dthetas[1]
    
    return (g,sigma)

# run MLE
(thetas, log_likelihood, log_likelihoods, thetass) = iterative_mle(obss,llog_p_T,update_thetas,options)

# plot
plt.plot(range(options['epochs']),log_likelihoods)
plt.show()
plt.plot(range(options['epochs']),thetass[0].reshape((thetass[0].shape[0],-1)))
plt.hlines(thetas_true[0].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
plt.plot(range(options['epochs']),thetass[1].reshape((thetass[1].shape[0],-1)))
plt.hlines(thetas_true[1].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
None

In [ ]:
%%time
# on S2
from src.statistics.mle import *

def llog_p_T(thetas,pars):
    (v,seed) = pars
    if seed:
        srng.seed(seed)
    g = thetas[0]
    G.sigma.set_value(thetas[1])
    return dlog_p_Tf(g,lift_to_fiber(v,x,G,M)[0])

def update_thetas(thetas, dthetas):
    g = thetas[0]
    sigma = thetas[1]
    
#     g = G.to_groupf(g+options['learning_rate']*dthetas[0])
    sigma += options['learning_rate']*dthetas[1]
    
    return (g,sigma)

# run MLE
(thetas, log_likelihood, log_likelihoods, thetass) = iterative_mle(obss_M,llog_p_T,update_thetas,options)

# plot
plt.plot(range(options['epochs']),log_likelihoods)
plt.show()
plt.plot(range(options['epochs']),thetass[0].reshape((thetass[0].shape[0],-1)))
plt.hlines(thetas_true[0].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
plt.plot(range(options['epochs']),thetass[1].reshape((thetass[1].shape[0],-1)))
plt.hlines(thetas_true[1].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
None